In [6]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine, Integer, String, Date, VARCHAR, text # import psycog2
from datetime import datetime,timedelta

In [ ]:
pasta_destino_json= r"C:\Users\lucas\OneDrive\Portfólio\Arquivo_Original"
df = pd.read_json(caminho_do_arquivo, encoding='utf-8-sig')##Esse comando serve para pegar o nosso arquivo json que e todo bagunçado e traduzir para o ecoding 'utf-8-sig'
df.head(3)## Visualiza e mostra na tela 
print("Salvo com sucesso!!")


NameError: name 'df' is not defined

In [ ]:
##Parâmetros de conexão
dbname =   'Port_Mineracao'
user =     'postgres'
password = '901012'
host =     'localhost'
port =     '5432'

conexao_str = f'postgresql://{user}:{password}@{host}:{port}/{dbname}'
engine = create_engine(conexao_str)

query = """ 
select 
"Matricula_FSTD",	
"Matriculas_Estrangeira",	
"AeronaveSimulada",	
"Nivel",	
"OperadorDoSimulador",	
"Endereco"
from public.anac_sqlalchemy

"""
df= pd.read_sql_query(query,engine)

NameError: name 'df' is not defined